In [1]:
import torch
import pickle
import numpy as np
from argparse import Namespace
from torch.utils.data import DataLoader
import torch.nn.functional as F
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import FineTuneEHRDataset, batcher
from heterogt.utils.train import train_with_early_stopping
from heterogt.utils.seed import set_random_seed

In [2]:
set_random_seed(123)

[INFO] Random seed set to 123


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
config = Namespace(
    dataset = "MIMIC-III",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"], 
    task_index = 2,
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]"],
    batch_size = 32,
    lr = 1e-3,
    epochs = 500,
    early_stop_patience = 5,   
)

In [5]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
curr_task = config.tasks[config.task_index]
print("Current task:", curr_task)
if curr_task == "next_diag_6m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_6m.pkl"
elif curr_task == "next_diag_12m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_12m.pkl"
else:
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_downstream.pkl"

Current task: stay


In [6]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_gender_sentences = ["[PAD]"] + [str(c) + "_" + gender for c in set(ehr_full_data["AGE"].values.tolist()) for gender in ["M", "F"]]
token_type_sentences = ["[PAD]"] + config.token_type
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [7]:
task_sentences = config.tasks
tokenizer = EHRTokenizer(token_type_sentences, age_gender_sentences, task_sentences, diag_sentences, 
                         med_sentences, lab_sentences, pro_sentences, special_tokens=config.special_tokens)
config.label_vocab_size = len(tokenizer.diag_voc.id2word)  # only for diagnosis
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_gender_vocab_size = tokenizer.token_number("age_gender")
print(f"Age and gender vocabulary size: {config.age_gender_vocab_size}")

Age and gender vocabulary size: 37


In [8]:
train_data, val_data, test_data = pickle.load(open(finetune_data_path, 'rb'))
# example label percentage
print("Percentage of DEATH in test dataset:",
      (test_data["DEATH"] == True).mean() * 100, "%")

print("Percentage of READMISSION in test dataset:",
      (test_data["READMISSION"] == 1).mean() * 100, "%")

print("Percentage of STAY>7 days in test dataset:",
      (test_data["STAY_DAYS"] > 7).mean() * 100, "%")

Percentage of DEATH in test dataset: 28.648477157360407 %
Percentage of READMISSION in test dataset: 40.1491116751269 %
Percentage of STAY>7 days in test dataset: 50.58692893401015 %


In [9]:
train_dataset = FineTuneEHRDataset(train_data, tokenizer, token_type=config.token_type, task=curr_task)
val_dataset = FineTuneEHRDataset(val_data, tokenizer, token_type=config.token_type, task=curr_task)
test_dataset = FineTuneEHRDataset(test_data, tokenizer, token_type=config.token_type, task=curr_task)

In [10]:
train_dataloader = DataLoader(
    train_dataset,
    collate_fn=batcher(tokenizer, config.task_index, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=True,
    batch_size=config.batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=batcher(tokenizer, config.task_index, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=batcher(tokenizer, config.task_index, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

In [11]:
if curr_task in ["death", "stay", "readmission"]:
    eval_metric = "prauc"
    task_type = "binary"
    loss_fn = F.binary_cross_entropy_with_logits
else:
    eval_metric = "prauc"
    task_type = "l2r"
    loss_fn = lambda x, y: F.binary_cross_entropy_with_logits(x, y)

In [12]:
input_ids, token_types, adm_index, age_gender_ids, task_index, labels = next(iter(train_dataloader))
print("Input IDs shape:", input_ids.shape)
print("Token Types shape:", token_types.shape)
print("Admission Index shape:", adm_index.shape)
print("Age/Sex IDs shape:", age_gender_ids.shape)
print("Task Index:", task_index)
print("Labels shape:", labels.shape)

Input IDs shape: torch.Size([32, 256])
Token Types shape: torch.Size([32, 256])
Admission Index shape: torch.Size([32, 256])
Age/Sex IDs shape: torch.Size([32, 7])
Task Index: 2
Labels shape: torch.Size([32, 1])


# Model Walkthrough

In [13]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData, Batch as HeteroBatch
from torch_geometric.nn import HeteroConv, GATConv

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [14]:
# class DiseaseOccHetGNN(nn.Module):
#     def __init__(self, d_model: int):
#         super().__init__()
#         self.conv1 = HeteroConv({
#             ('visit','contains','occ'): GATConv(d_model, d_model, add_self_loops=False),
#             ('occ','contained_by','visit'): GATConv(d_model, d_model, add_self_loops=False),
#             ('visit','next','visit'): GATConv(d_model, d_model, add_self_loops=False),
#         }, aggr='mean')
#         self.lin = nn.Linear(d_model, d_model)
    
#     def forward(self, hg: HeteroData):
#         # x_dict: {'visit': [N_visit, d], 'occ': [N_occ, d]}
#         x_dict = {'visit': hg['visit'].x, 'occ': hg['occ'].x}
#         x_dict = self.conv1(x_dict, hg.edge_index_dict)
#         x_dict = {k: self.lin(v) for k, v in x_dict.items()}
#         return x_dict # {'visit': [N_visit, d], 'occ': [N_occ, d]}


class DiseaseOccHetGNN(nn.Module):
    def __init__(self, d_model: int, heads: int = 1, dropout: float = 0.0):
        super().__init__()
        self.act = nn.GELU()

        # 第1层
        self.conv1 = HeteroConv({
            ('visit','contains','occ'): GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
            ('occ','contained_by','visit'): GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
            ('visit','next','visit'):       GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
        }, aggr='mean')

        # 第2层
        self.conv2 = HeteroConv({
            ('visit','contains','occ'): GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
            ('occ','contained_by','visit'): GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
            ('visit','next','visit'):       GATConv(d_model, d_model, heads=heads, concat=False, add_self_loops=False),
        }, aggr='mean')
        self.lin = nn.Linear(d_model, d_model)

    def forward(self, hg):
        # x_dict: {'visit': [N_visit, d], 'occ': [N_occ, d]}
        x_dict = {'visit': hg['visit'].x, 'occ': hg['occ'].x}

        # 第1层：HeteroConv → Linear → GELU → Dropout
        x_dict = self.conv1(x_dict, hg.edge_index_dict)
        # x_dict = {k: self.act(v)   for k, v in x_dict.items()}

        # 第2层：HeteroConv → Linear（末层通常不再加激活/随你需要）
        x_dict = self.conv2(x_dict, hg.edge_index_dict)
        x_dict = {k: self.lin(v) for k, v in x_dict.items()}

        return x_dict  # {'visit': [N_visit, d], 'occ': [N_occ, d]}

In [15]:
# multi-class classification task
class MultiPredictionHead(nn.Module):
    def __init__(self, hidden_size, label_size):
        super(MultiPredictionHead, self).__init__()
        self.cls = nn.Sequential(
                nn.Linear(hidden_size, hidden_size), 
                nn.ReLU(), 
                nn.Linear(hidden_size, label_size)
            )

    def forward(self, input):
        return self.cls(input)
    
class BinaryPredictionHead(nn.Module):
    def __init__(self, hidden_size):
        super(BinaryPredictionHead, self).__init__()
        self.cls = nn.Sequential(
                nn.Linear(hidden_size, hidden_size), 
                nn.ReLU(), 
                nn.Linear(hidden_size, 1)
            )
    def forward(self, input):
        return self.cls(input)

In [16]:
for i in range(len(train_dataset)):
    age_gender_ids = train_dataset[i][3]
    if len(age_gender_ids[0]) > 3:
        print(age_gender_ids)
        break
exp_i = i
id_seq = torch.concat([train_dataset[exp_i][0][0], torch.zeros(5, dtype=train_dataset[exp_i][0][0].dtype)], dim=0)
type_seq = torch.concat([train_dataset[exp_i][1][0], torch.zeros(5, dtype=train_dataset[exp_i][1][0].dtype)], dim=0)
visit_seq = torch.concat([train_dataset[exp_i][2][0], torch.zeros(5, dtype=train_dataset[exp_i][2][0].dtype)], dim=0)
age_sex = torch.concat([train_dataset[exp_i][3][0], torch.zeros(3, dtype=train_dataset[exp_i][3][0].dtype)], dim=0)

tensor([[17, 17, 17,  7]])


In [ ]:
class HeteroGT(nn.Module):
    def __init__(self, tokenizer, d_model, num_heads, num_layers, max_num_adms, device, task, use_hetero_graph):
        super(HeteroGT, self).__init__()
        self.device = device
        self.tokenizer = tokenizer
        self.max_num_adms = max_num_adms
        self.use_hetero_graph = use_hetero_graph
        self.global_vocab_size = len(self.tokenizer.vocab.word2id)
        self.age_sex_vocab_size = len(self.tokenizer.age_gender_voc.word2id)
        self.n_type = len(self.tokenizer.token_type_voc.word2id)
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.seq_pad_id = tokenizer.convert_tokens_to_ids(["[PAD]"], voc_type="all")[0] #0
        self.type_pad_id = tokenizer.convert_tokens_to_ids(["[PAD]"], voc_type="type")[0] #0
        self.adm_pad_id = 0
        self.age_sex_pad_id = tokenizer.convert_tokens_to_ids(["[PAD]"], voc_type="age_gender")[0] #0
        self.diag_type_id = 1
        self.visit_type_id = 5
        
        # embedding layers
        self.token_emb = nn.Embedding(self.global_vocab_size, d_model, padding_idx=self.seq_pad_id)
        self.type_emb = nn.Embedding(self.n_type + 1, d_model, padding_idx=self.type_pad_id)
        self.adm_index_emb = nn.Embedding(self.max_num_adms + 1, d_model, padding_idx=self.adm_pad_id) # +1 for pad
        self.age_sex_emb = nn.Embedding(self.age_sex_vocab_size, d_model, padding_idx=self.age_sex_pad_id)
        self.task_emb = nn.Embedding(5, d_model, padding_idx=None)  # task embedding, not used in this model
        
        # GNN
        self.het_gnn = DiseaseOccHetGNN(d_model)    

        # encoder transformer
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, batch_first=True, norm_first = True)
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers, enable_nested_tensor=False)

        # prediction head
        if task in ["death", "stay", "readmission"]:
            self.cls_head = BinaryPredictionHead(self.d_model)
        else:
            self.cls_head = MultiPredictionHead(self.d_model, config.label_vocab_size)

    def forward(self, input_ids, token_types, adm_index, age_gender_index, task_id):
        B, L = input_ids.shape
        task_id = torch.full((B,), task_id, dtype=torch.long, device=self.device)
        # 基础表示
        token_embed = self.token_emb(input_ids)  # [B, L, d]
        adm_emb  = self.adm_index_emb(adm_index)          # [B, L, d]
        type_emb = self.type_emb(token_types)       # [B, L, d]
        x_tokens = token_embed + adm_emb + type_emb  # [B, L, d]
        task_emb = self.task_emb(task_id).unsqueeze(1)           # [B, 1, d]
        x = torch.cat([task_emb, x_tokens], dim=1)  # [B, 1+L, d]
        
        # mask      
        seq_pad_mask = (input_ids == self.seq_pad_id)         # [B, L]
        task_pad_mask = torch.zeros((B, 1), dtype=torch.bool, device=self.device)
        mask = torch.concat([task_pad_mask, seq_pad_mask], dim=1)  # [B, 1+L]
        
        if self.use_hetero_graph:
            # get visit embed and mask
            visit_emb_pad, visit_pad_mask = self.visit_segment(B, input_ids, token_types, adm_index, age_gender_index)
            x = torch.cat([x, visit_emb_pad], dim=1)  # [B, 1+L(+V), d]
            mask = torch.concat([mask, visit_pad_mask], dim=1)

        # ===== Transformer 编码（batch_first=True） =====
        h = self.encoder(x, src_key_padding_mask=mask)   # [B, 1+L(+V), d]

        # ===== 分类：取 CLS（task 位） =====
        logits = self.cls_head(h[:, 0, :])  # [B, label_size]
        return logits

    def visit_segment(self, B, input_ids, token_types, adm_index, age_gender_index):
        graphs = []
        for p in range(B):
            hg_p = self.build_patient_graph(input_ids[p], token_types[p], adm_index[p], age_gender_index[p])
            graphs.append(hg_p)

        batch_graph = HeteroBatch.from_data_list(graphs).to(self.device)
        out = self.het_gnn(batch_graph)
        h_visit_all = out['visit']  # extract virtual visit node representations

        # 取出每个样本的 visit 表示序列（按我们在 build_patient_graph 中的保序构造）
        visit_emb_seq = []
        offset = 0
        for p in range(B):
            n_v = graphs[p]['visit'].num_nodes
            visit_emb_p = h_visit_all[offset:offset + n_v]  # [N_visit_p, d]
            offset += n_v
            visit_emb_seq.append(visit_emb_p)
            
        visit_emb_pad = []
        visit_pad_mask = []
        visit_index_pad = []
        for p in range(B):
            v = visit_emb_seq[p]                          # [N_visit_p, d]
            Np = v.size(0)
            if Np < self.max_num_adms:
                pad_len = self.max_num_adms - Np
                v_pad = torch.cat([v, torch.zeros(pad_len, self.d_model, device=self.device, dtype=v.dtype)], dim=0)
                m_pad = torch.cat([torch.zeros(Np, dtype=torch.bool, device=self.device), torch.ones(pad_len, dtype=torch.bool, device=self.device)], dim=0)
                i_pad = torch.cat([torch.arange(1, Np + 1, device=self.device), torch.full((pad_len,), self.adm_pad_id, dtype=torch.long, device=self.device)], dim=0)
            else:
                v_pad = v[:self.max_num_adms]
                m_pad = torch.zeros(self.max_num_adms, dtype=torch.bool, device=self.device)
                i_pad = torch.arange(1, self.max_num_adms + 1, device=self.device)
            visit_emb_pad.append(v_pad)      # [V_max, d]
            visit_pad_mask.append(m_pad)     # [V_max]
            visit_index_pad.append(i_pad)    # [V_max]

        visit_emb_pad  = torch.stack(visit_emb_pad,  dim=0)  # [B, V_max, d]
        visit_pad_mask = torch.stack(visit_pad_mask, dim=0)  # [B, V_max]
        visit_index_pad = torch.stack(visit_index_pad, dim=0)  # [B, V_max]

        # ====== 对齐与类型嵌入（关键部分） ======
        nonpad = (~visit_pad_mask).unsqueeze(-1)             # [B, V_max, 1], bool

        # 1. 加 type embedding（仅非 pad 位）
        visit_type_ids = torch.full((B, self.max_num_adms), self.visit_type_id, dtype=torch.long, device=self.device) # [B, V_max]                                                   # [B, V_max]
        visit_type_emb = self.type_emb(visit_type_ids) * nonpad      # [B, V_max, d]
        
        # 2. 加 visit index embedding（仅非 pad 位）
        visit_index_emb = self.adm_index_emb(visit_index_pad) * nonpad  # [B, V_max, d]
        
        # 3. 得到最终embedding
        visit_emb_pad  = visit_emb_pad + visit_type_emb + visit_index_emb

        return visit_emb_pad, visit_pad_mask
    
    def build_patient_graph(self, id_seq: torch.Tensor, type_seq: torch.Tensor, visit_seq: torch.Tensor, age_sex: torch.Tensor):
        # build a graph just for one patient
        hg = HeteroData()
        occ_mask = (type_seq == self.diag_type_id) & (id_seq != self.seq_pad_id) # 疾病token mask
        occ_pos = torch.nonzero(occ_mask, as_tuple=False).view(-1) # 疾病 token 的位置索引，形状 [N_occ]
        N_occ = occ_pos.numel() # 疾病 token 数量

        # build visit virtual nodes
        nonpad = id_seq != self.seq_pad_id
        visit_used = visit_seq[nonpad] # seq非pad部分
        visit_ids_unique, visit_lid_nonpad = torch.unique(visit_used, return_inverse=True)
        visit_lid_full = torch.full_like(id_seq, fill_value=-1)
        visit_lid_full[nonpad] = visit_lid_nonpad
        N_visit = visit_ids_unique.numel()
        age_sex_nonpad = age_sex[age_sex!=self.age_sex_pad_id]
        assert N_visit == len(visit_ids_unique) == len(age_sex_nonpad)
        visit_x = self.age_sex_emb(age_sex_nonpad.to(self.device))
        hg['visit'].x = visit_x
        hg['visit'].num_nodes = N_visit
        
        # build diag nodes
        gid_occ = id_seq[occ_pos]
        x_occ = self.token_emb(gid_occ) # [N_occ, d]
        hg['occ'].x = x_occ
        hg['occ'].num_nodes = N_occ

        # build edges between diag nodes and virtual visit nodes
        occ_visit_lid = visit_lid_full[occ_pos]
        e_v2o = torch.stack([occ_visit_lid, torch.arange(N_occ, device=self.device)], dim=0)
        e_o2v = torch.stack([torch.arange(N_occ, device=self.device), occ_visit_lid], dim=0)
        hg['visit','contains','occ'].edge_index = e_v2o
        hg['occ','contained_by','visit'].edge_index = e_o2v
        
        # build forward edges between virtual visit nodes
        if N_visit > 1:
            src = torch.arange(0, N_visit - 1, device=self.device)
            dst = torch.arange(1, N_visit, device=self.device)
            e_next = torch.stack([src, dst], dim=0) # [2, N_visit-1]
        else:
            e_next = torch.empty(2, 0, dtype=torch.long, device=self.device)
        hg['visit','next','visit'].edge_index = e_next
        return hg

In [18]:
final_metrics = {"precision":[],"recall":[],"f1":[],"auc":[],"prauc":[]}
for i in range(5):
    model = HeteroGT(tokenizer, d_model=128, num_heads=4, num_layers=2, max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, use_hetero_graph=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                             optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                             val_long_seq_idx=None, test_long_seq_idx=None, eval_metric=eval_metric, return_model=False)
    for key in final_metrics.keys():
        final_metrics[key].append(best_test_metric[key])

Running inference: 100%|██████████| 197/197 [00:02<00:00, 66.49it/s]


Validation: {'precision': 0.7512324611272233, 'recall': 0.6211978676681681, 'f1': 0.6800549212354794, 'auc': 0.776896440614259, 'prauc': 0.7812321168399854}
Test:      {'precision': 0.7332116788294408, 'recall': 0.6299780495433366, 'f1': 0.6776859454396119, 'auc': 0.7655372324709688, 'prauc': 0.7670472141096356}


Running inference: 100%|██████████| 197/197 [00:02<00:00, 66.44it/s]


Validation: {'precision': 0.8185966913815255, 'recall': 0.44998432110238323, 'f1': 0.5807365393291547, 'auc': 0.7889364363180105, 'prauc': 0.7975717189829002}
Test:      {'precision': 0.8121964382039277, 'recall': 0.4719347757903044, 'f1': 0.5969853186330997, 'auc': 0.7804364622168801, 'prauc': 0.7907392164579675}


Running inference: 100%|██████████| 197/197 [00:02<00:00, 66.81it/s]


...

Validation: {'precision': 0.7264739229004352, 'recall': 0.8037002195020266, 'f1': 0.7631383008017393, 'auc': 0.8313227070506002, 'prauc': 0.8348720418947138}
Test:      {'precision': 0.7176536626418253, 'recall': 0.8018187519573478, 'f1': 0.7574052082832395, 'auc': 0.8233132351527397, 'prauc': 0.8281372644814688}


Running inference: 100%|██████████| 197/197 [00:03<00:00, 61.35it/s]


Validation: {'precision': 0.708913649023095, 'recall': 0.7980558168679898, 'f1': 0.7508482027155312, 'auc': 0.8158642497957121, 'prauc': 0.8148847129275961}
Test:      {'precision': 0.7035287579863754, 'recall': 0.7939793038545188, 'f1': 0.746022387473321, 'auc': 0.8118871703342196, 'prauc': 0.8130526419030113}


Running inference: 100%|██████████| 197/197 [00:02<00:00, 67.19it/s]

Validation: {'precision': 0.689646335210944, 'recall': 0.843838193788511, 'f1': 0.758990264403932, 'auc': 0.8244369035702168, 'prauc': 0.827649992290554}
Test:      {'precision': 0.6825599184072347, 'recall': 0.8394481028509269, 'f1': 0.7529180093949839, 'auc': 0.8192846396647384, 'prauc': 0.8247632884110783}

Early stopping triggered (no improvement for 5 epochs).

Best validation performance:
{'precision': 0.7429837518441862, 'recall': 0.788648479144595, 'f1': 0.7651353768092357, 'auc': 0.834663904448054, 'prauc': 0.842451013613768}
Corresponding test performance:
{'precision': 0.7314652656137436, 'recall': 0.7858262778275766, 'f1': 0.7576719526760851, 'auc': 0.8298115965445022, 'prauc': 0.8384766131054688}


In [19]:
# print the mean and std of the final metrics
print("\nFinal Metrics:")
for key in final_metrics.keys():
    mean_value = np.mean(final_metrics[key])
    std_value = np.std(final_metrics[key])
    print(f"{key}: {mean_value:.4f} ± {std_value:.4f}")


Final Metrics:
precision: 0.7333 ± 0.0172
recall: 0.7531 ± 0.0461
f1: 0.7417 ± 0.0159
auc: 0.8174 ± 0.0080
prauc: 0.8278 ± 0.0078


In [ ]:
final_metrics = {"precision":[],"recall":[],"f1":[],"auc":[],"prauc":[]}
for i in range(5):
    model = HeteroGT(tokenizer, d_model=128, num_heads=4, num_layers=2, max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, use_hetero_graph=False).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                             optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                             val_long_seq_idx=None, test_long_seq_idx=None, eval_metric=eval_metric, return_model=False)
    for key in final_metrics.keys():
        final_metrics[key].append(best_test_metric[key])

Epoch 001:   0%|          | 0/98 [00:00<?, ?it/s, loss=0.7284]

Running inference: 100%|██████████| 197/197 [00:00<00:00, 320.37it/s]


Validation: {'precision': 0.649757281551821, 'recall': 0.8394481028509269, 'f1': 0.7325215438546027, 'auc': 0.7838736247809551, 'prauc': 0.7941442224893596}
Test:      {'precision': 0.6478017974237362, 'recall': 0.8363123236097952, 'f1': 0.73008485683613, 'auc': 0.7779395162041266, 'prauc': 0.7890250631825644}


Running inference: 100%|██████████| 197/197 [00:00<00:00, 318.65it/s]


Validation: {'precision': 0.8079019073532793, 'recall': 0.55785512699731, 'f1': 0.6599888655046029, 'auc': 0.7987582274015663, 'prauc': 0.8042098917250383}
Test:      {'precision': 0.7897321428536173, 'recall': 0.5547193477561784, 'f1': 0.651685388408805, 'auc': 0.7899071195275493, 'prauc': 0.79772077041645}


...

Running inference: 100%|██████████| 197/197 [00:00<00:00, 318.78it/s]


Validation: {'precision': 0.725076452597171, 'recall': 0.7434932580723002, 'f1': 0.7341693710629192, 'auc': 0.8111218986203877, 'prauc': 0.8132447702401917}
Test:      {'precision': 0.7299999999977879, 'recall': 0.7554092191886002, 'f1': 0.7424872811757812, 'auc': 0.8154954304700095, 'prauc': 0.821061593542323}


Running inference: 100%|██████████| 197/197 [00:00<00:00, 317.45it/s]


Validation: {'precision': 0.6786079836216004, 'recall': 0.8316086547480979, 'f1': 0.7473580336570754, 'auc': 0.8004029290448815, 'prauc': 0.7977663463071984}
Test:      {'precision': 0.6891578676263478, 'recall': 0.8391345249268136, 'f1': 0.7567873253627527, 'auc': 0.8125188562006134, 'prauc': 0.8117823903357132}

Early stopping triggered (no improvement for 5 epochs).

Best validation performance:
{'precision': 0.8380864765370833, 'recall': 0.5713389777341759, 'f1': 0.6794704408226561, 'auc': 0.8296376652802323, 'prauc': 0.840215561367446}
Corresponding test performance:
{'precision': 0.8382616487417641, 'recall': 0.5867042960157206, 'f1': 0.690278541546932, 'auc': 0.8282272478579304, 'prauc': 0.8426821693031619}


Epoch 001:   9%|▉         | 9/98 [00:00<00:01, 83.19it/s, loss=0.7705]

In [ ]:
# print the mean and std of the final metrics
print("\nFinal Metrics:")
for key in final_metrics.keys():
    mean_value = np.mean(final_metrics[key])
    std_value = np.std(final_metrics[key])
    print(f"{key}: {mean_value:.4f} ± {std_value:.4f}")


Final Metrics:
precision: 0.7711 ± 0.0397
recall: 0.6721 ± 0.0880
f1: 0.7122 ± 0.0370
auc: 0.8141 ± 0.0069
prauc: 0.8249 ± 0.0061
